<a href="https://colab.research.google.com/github/karthik-sriram-5503/website-traffic-analysis/blob/main/ML_PREDICTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
import xgboost as xg

In [ ]:
df=pd.read_csv('/content/daily-website-visitors.csv')

df.rename(columns = {'Day.Of.Week':'day_of_week'
                    ,'Page.Loads':'page_loads'
                    ,'Unique.Visits':'unique_visits'
                    ,'First.Time.Visits':'first_visits'
                    ,'Returning.Visits':'returning_visits'}, inplace = True)

df=df.replace(',','',regex=True)

df['page_loads']=df['page_loads'].astype(int)
df['unique_visits']=df['unique_visits'].astype(int)
df['first_visits']=df['first_visits'].astype(int)
df['returning_visits']=df['returning_visits'].astype(int)

df

,Row,Day,day_of_week,Date,page_loads,unique_visits,first_visits,returning_visits
0,1,Sunday,1,9/14/2014,2146,1582,1430,152
1,2,Monday,2,9/15/2014,3621,2528,2297,231
2,3,Tuesday,3,9/16/2014,3698,2630,2352,278
3,4,Wednesday,4,9/17/2014,3667,2614,2327,287
4,5,Thursday,5,9/18/2014,3316,2366,2130,236
...,...,...,...,...,...,...,...,...
2162,2163,Saturday,7,8/15/2020,2221,1696,1373,323
2163,2164,Sunday,1,8/16/2020,2724,2037,1686,351
2164,2165,Monday,2,8/17/2020,3456,2638,2181,457
2165,2166,Tuesday,3,8/18/2020,3581,2683,2184,499


In [ ]:
pred_df=df[['page_loads' ,'unique_visits' ,'first_visits' ,'returning_visits','Day']]

In [ ]:
pred_df['days_f']=np.where((df['Day']=='Tuesday') |
                      (df['Day']=='Wednesday') |
                      (df['Day']=='Thursday') |
                      (df['Day']=='Monday'),1,0)

pred_df

<ipython-input-5-d5891787bde2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pred_df['days_f']=np.where((df['Day']=='Tuesday') |


,page_loads,unique_visits,first_visits,returning_visits,Day,days_f
0,2146,1582,1430,152,Sunday,0
1,3621,2528,2297,231,Monday,1
2,3698,2630,2352,278,Tuesday,1
3,3667,2614,2327,287,Wednesday,1
4,3316,2366,2130,236,Thursday,1
...,...,...,...,...,...,...
2162,2221,1696,1373,323,Saturday,0
2163,2724,2037,1686,351,Sunday,0
2164,3456,2638,2181,457,Monday,1
2165,3581,2683,2184,499,Tuesday,1


In [ ]:
pred_df.drop('Day',axis=1,inplace=True)

<ipython-input-6-02139acee9f0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pred_df.drop('Day',axis=1,inplace=True)


In [ ]:
X2=pred_df[['page_loads','first_visits' ,'returning_visits','days_f']]
y2=pred_df['unique_visits']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X2, y2, test_size=0.3,random_state=42)

In [ ]:
from sklearn.linear_model import LinearRegression
regressor2 = LinearRegression(fit_intercept=False)
regressor2.fit(X_train, y_train)

LinearRegression(fit_intercept=False)

In [ ]:
y_pred2 = regressor2.predict(X_test)

In [ ]:
lr2 = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred2})
lr2

,Actual,Predicted
1486,4173,4173.0
1602,1902,1902.0
1460,2870,2870.0
1134,2142,2142.0
1513,4329,4329.0
...,...,...
439,2579,2579.0
271,2494,2494.0
244,1818,1818.0
1159,3332,3332.0


In [ ]:
dtr = DecisionTreeRegressor(random_state=0)
dtr.fit(X_train, y_train)

DecisionTreeRegressor(random_state=0)

In [ ]:
dtr_pred = dtr.predict(X_test)

In [ ]:
dtr_g = pd.DataFrame({'Actual': y_test, 'Predicted': dtr_pred})
dtr_g

,Actual,Predicted
1486,4173,4140.0
1602,1902,1929.0
1460,2870,2871.0
1134,2142,2198.0
1513,4329,4330.0
...,...,...
439,2579,2572.0
271,2494,2518.0
244,1818,1826.0
1159,3332,3341.0


In [ ]:
dtr.score(X_test,y_test)*100

99.85504139672305